# Image_to_Shapefile
This script is used to clip your bathymetry file to desired depth, 150m and 80m was chosen to include onshore and offshore wind, from an Image server. 

In [1]:
!pip install arcgis
!pip install rasterio
!pip install geopandas

In [2]:
from arcgis.raster import ImageryLayer
from arcgis.gis import GIS
from rasterio.transform import from_bounds
from rasterio.features import shapes
from shapely.geometry import shape
from itertools import product
import multiprocessing as mp
import geopandas as gpd
import numpy as np
import os
import pathlib

# Get the user's Downloads folder
downloads_path = str(pathlib.Path.home() / "Downloads")


def worker(args):
    ''' Tasks to be carried out by each CPU for parallelization '''
    j, arg = args
    i, x = arg
    lat = np.arange( 48, 57, .05)

    url = "https://image.marine.ie/arcgis/rest/services/INFOMAR/AllSurvey/ImageServer" ######## Specify URL here
    img_layer = ImageryLayer(url)
    
    point = {"x": x, "y": lat[j], "spatialReference": {"wkid": 4326}}

    try:
        # Retrieve bathymetry for this longitude-latitude point
        sample = img_layer.get_samples(geometry=point, 
                                       geometry_type='esriGeometryPoint',  
                                       return_first_value_only=True)
        mpdata[j] = float(sample[0]['value'])
    except Exception:
        mpdata[j] = -999999
    
def init(A):
    global mpdata
    mpdata = A

def main():
    ''' Define raster grid '''
    lon = np.arange(-12, -5, .05); L = len(lon) # Longitude
    lat = np.arange( 48, 57, .05); M = len(lat) # Latitude
    # Initialize bathymetry
    H = -999999*np.ones((L, M))
    
    # Shared-memory array
    mpdata = mp.Array('d', [1]*M)
    
    for i, x in enumerate(lon):
        print('Procesing longitude %.2f' % x)
        # Wrap arguments needed by the workers (longitude)
        args = (i, x)
        
        # Start parallel loop
        pool = mp.Pool(initializer=init, initargs=(mpdata,))
        pool.imap(worker, product(range(M,), (args,)) )
        pool.close(); pool.join()

        # Write bathymetry
        H[i, :] = mpdata
        
    create_shapefile_from_raster(lon, lat, H)

def create_shapefile_from_raster(lon, lat, H):
    ''' Convert gridded bathymetry into shapefile '''

    print('Creating shapefile...')

    # Preparing  data for shapefile creation
    data = H.T; longitude = np.squeeze(lon); latitude = np.squeeze(lat)

    # This seems to be required to get the right orientation of the bathymetry
    data = np.flip(data, axis=0)

    # Mask below 150 meters and missing values
    mask = np.logical_or(data > 100, data < -80); masked_array = np.ma.masked_array(data, mask=mask)

    L, M = masked_array.shape

    # Create affine transform from bounds
    transform = from_bounds(longitude.min(),
                            latitude.min(),
                            longitude.max(),
                            latitude.max(), M, L)

    # Binary mask: 1 for unmasked, 0 for masked
    binary_mask = (~masked_array.mask).astype(np.uint8)

    # Extract polygons
    polygons = [shape(geom) for geom, value in shapes(binary_mask, mask=None, transform=transform) if value == 1]

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

    # Save to shapefile
    #os.makedirs("INFOMAR-150", exist_ok=True)

    # Create a subfolder for your output
    output_folder = os.path.join(downloads_path, "INFOMAR-80")
    os.makedirs(output_folder, exist_ok=True)

    # Full path to shapefile
    output_path = os.path.join(output_folder, "INFOMAR-80.shp")

    # Save to Downloads
    gdf.to_file(output_path)

    #print(f"Shapefile saved to: {output_path}")
    #gdf.to_file("INFOMAR-150/INFOMAR-150.shp") ############################################################ GL to save to S3 #####################################################################

    print('INFOMAR-80.shp created')

    return output_folder

    from upload_model import upload_model_to_s3_bucket
    upload_model_to_s3_bucket(output_folder)
    print("Upload finished.")




######## Extra cell below to perform the same function but only converting image to raster to shapefile - no masking.

## Download to S3 Bucket

In [ ]:
import os

dir_output = os.path.abspath(pwd) #f'./data'
print(dir_output)

from upload_model import upload_model_to_s3_bucket

upload_model_to_s3_bucket(dir_output)

## Download to Local Machine

In [ ]:
import base64
from IPython.display import HTML

zip_path = "/home/jovyan/Downloads/INFOMAR-80.zip"

# Read the ZIP file
with open(zip_path, "rb") as f:
    data = f.read()

# Encode the file in Base64 and build a browser download link
b64 = base64.b64encode(data).decode()
payload = f"data:application/zip;base64,{b64}"

html = f"""
<a href="{payload}" download="INFOMAR-80.zip"
   style="background: #007bff; color: white; padding: 10px 16px; 
          border-radius: 6px; text-decoration: none; font-weight: bold;">
   ⬇️ Click here to download INFOMAR-80.zip
</a>
"""

display(HTML(html))
